# Detecção de Bots Dataset Telegram

Assumimos que bots apresentam comportamento coordenado com outros usuários ou bots. Utilizando hipóteses de comportamento humano, serão criados grafos com candidatos a bots que serão filtrados usando algoritmos de comunidade. As hipóteses definidas foram:

1. Pessoas dificilmente compartilham mensagens poucos segundos após ser enviada
2. Pessoas dificilmente compartilham mensagens no mesmo horário frequentemente
3. Pessoas apresentam intervalos de tempo entre mensagens caóticos. Bots tendem a enviar mensagens de forma mais sistematizada

In [1]:
# dependências
import networkx as nx
from networkx.algorithms import community
import concurrent.futures
import os

import warnings
import os
os.chdir("..")
from util.load_graph import load_graph_by_edge, get_driver
from util.bot_filtering import rank_bot_suspicion

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
driver = get_driver()

## Hipótese 1: Usuários que compartilham mensagens rápidos demais

![rapid share](../../images/rapid_share.png)

In [3]:
G_rapid = load_graph_by_edge(driver, 'RAPID_SHARE')

rapid_bots = rank_bot_suspicion(G_rapid) 
rapid_bots

,user_id,bot_suspicion_score,centrality_rank,clique_score,volume_score,labels
24,c743967449a387ad2c1c7e03b2c45b36,0.6670,0.1674,1.0000,1.0000,[]
23,1665e22b0f564cd46d343f7677014821,0.5835,0.1259,0.9709,0.7241,[]
26,b4ce87d44421ef3d2c4710051df5fe40,0.5151,0.0772,1.0000,0.4212,[]
157,1ac091b8ed5c4e42383f1b4ff4cc9b2d,0.4150,1.0000,0.0002,0.0743,[]
0,abe534d581ec6d552243d6955d3c3cd8,0.2347,0.5513,0.0000,0.0709,[]
...,...,...,...,...,...,...
178,761cf2c6592fbabbf0dae751e5c36a05,0.0092,0.0224,0.0000,0.0011,[]
245,942d61531313972a81171f1d87d80ac0,0.0089,0.0218,0.0000,0.0011,[]
73,e04f82940a5de7979fffadb350488924,0.0085,0.0207,0.0000,0.0011,[]
103,584b05446c0afec7298ccd5c0a11fa0a,0.0079,0.0193,0.0000,0.0011,[]


In [4]:
rapid_bots.describe()

,bot_suspicion_score,centrality_rank,clique_score,volume_score
count,474.000000,474.000000,474.000000,474.000000
mean,0.036305,0.078912,0.007692,0.008271
std,0.051158,0.060814,0.078742,0.060019
min,0.005000,0.012000,0.000000,0.001100
25%,0.020800,0.049025,0.000000,0.001100
50%,0.031800,0.078900,0.000000,0.002300
75%,0.034800,0.078900,0.000000,0.003400
max,0.667000,1.000000,1.000000,1.000000


In [5]:
filtered_rapid_bots = rapid_bots[
    rapid_bots['bot_suspicion_score'] >
    rapid_bots['bot_suspicion_score'].quantile(0.8)
]
filtered_rapid_bots

,user_id,bot_suspicion_score,centrality_rank,clique_score,volume_score,labels
24,c743967449a387ad2c1c7e03b2c45b36,0.6670,0.1674,1.0000,1.0000,[]
23,1665e22b0f564cd46d343f7677014821,0.5835,0.1259,0.9709,0.7241,[]
26,b4ce87d44421ef3d2c4710051df5fe40,0.5151,0.0772,1.0000,0.4212,[]
157,1ac091b8ed5c4e42383f1b4ff4cc9b2d,0.4150,1.0000,0.0002,0.0743,[]
0,abe534d581ec6d552243d6955d3c3cd8,0.2347,0.5513,0.0000,0.0709,[]
...,...,...,...,...,...,...
155,e3afed0047b08059d0fada10f400c1e5,0.0403,0.0947,0.0043,0.0034,[]
29,d49f81df0c75d1d72bee6c5b2d707da0,0.0403,0.0122,0.0874,0.0023,[]
177,50cc2b20a3c2c24ce68bb2c0554ddae1,0.0398,0.0874,0.0081,0.0079,[]
386,96c8dffb29c65cd1905239dc64196f03,0.0397,0.0958,0.0000,0.0068,[]


## Hipótese 2: Usuários que compartilham mensagens no mesmo horário frequentemente

![hourly](../../images/hourly.png)

In [6]:
G_hourly = load_graph_by_edge(driver, 'HOURLY_SHARED')

hourly_bots = rank_bot_suspicion(G_hourly) 
hourly_bots

,user_id,bot_suspicion_score,centrality_rank,clique_score,volume_score,labels
300,c743967449a387ad2c1c7e03b2c45b36,0.6131,0.0327,1.0000,1.0000,[]
302,b4ce87d44421ef3d2c4710051df5fe40,0.5278,0.0201,0.9909,0.6171,[]
129,3f64a652deccf6273d7d769252bdfb46,0.4119,1.0000,0.0007,0.0579,[]
416,1ac091b8ed5c4e42383f1b4ff4cc9b2d,0.3799,0.9319,0.0000,0.0357,[]
298,1665e22b0f564cd46d343f7677014821,0.3300,0.0226,0.4993,0.6058,[]
...,...,...,...,...,...,...
963,ba7730700b6383ccca7aa7b7283662c2,0.0010,0.0026,0.0000,0.0000,[]
897,283962e68cad3228788119077c54d252,0.0010,0.0025,0.0000,0.0000,[]
843,b417646992e5d8eaf52aab1425b316ac,0.0010,0.0025,0.0000,0.0000,[]
1815,2daf1eadf52eff54e2267af25e0c1813,0.0010,0.0026,0.0000,0.0000,[]


In [7]:
hourly_bots.describe()

,bot_suspicion_score,centrality_rank,clique_score,volume_score
count,2413.000000,2413.000000,2413.000000,2413.000000
mean,0.007069,0.015684,0.001237,0.001485
std,0.023523,0.037810,0.030400,0.027115
min,0.000900,0.002400,0.000000,0.000000
25%,0.002100,0.004900,0.000000,0.000000
50%,0.003600,0.008700,0.000100,0.000100
75%,0.006300,0.015700,0.000200,0.000300
max,0.613100,1.000000,1.000000,1.000000


In [8]:
filtered_hourly_bots = hourly_bots[
    hourly_bots['bot_suspicion_score'] >
    hourly_bots['bot_suspicion_score'].quantile(0.8)
]
filtered_hourly_bots

,user_id,bot_suspicion_score,centrality_rank,clique_score,volume_score,labels
300,c743967449a387ad2c1c7e03b2c45b36,0.6131,0.0327,1.0000,1.0000,[]
302,b4ce87d44421ef3d2c4710051df5fe40,0.5278,0.0201,0.9909,0.6171,[]
129,3f64a652deccf6273d7d769252bdfb46,0.4119,1.0000,0.0007,0.0579,[]
416,1ac091b8ed5c4e42383f1b4ff4cc9b2d,0.3799,0.9319,0.0000,0.0357,[]
298,1665e22b0f564cd46d343f7677014821,0.3300,0.0226,0.4993,0.6058,[]
...,...,...,...,...,...,...
1333,2925420ed6e32dcf61a65bc18c8fdbca,0.0067,0.0164,0.0003,0.0004,[]
121,f28f867a29cb2ffd97768829f7e57943,0.0067,0.0165,0.0001,0.0004,[]
501,7bf3bc465d958353740236d1b3770cb7,0.0067,0.0164,0.0003,0.0004,[]
506,8d5588a812455beddafec9db8dfb255a,0.0067,0.0164,0.0003,0.0004,[]


## Hipótese 3: Usuários que compartilham mensagens com pouca variação
![hourly](../../images/metronome.png)

In [9]:
G_metronome = load_graph_by_edge(driver, 'METRONOME_SIMILAR')

metronome_bots = rank_bot_suspicion(G_metronome) 
metronome_bots

,user_id,bot_suspicion_score,centrality_rank,clique_score,volume_score,labels
38,96dd9aa3ec0b34b3960f2e1e939a8d64,0.8308,0.8351,0.7418,1.0000,[]
1620,07916b5f9d395b4ba287939d7662b94d,0.8289,0.8296,0.7461,0.9930,[]
137,6106f6a23e4332847cbafa9f5bcccdbd,0.8289,0.8297,0.7459,0.9929,[]
1593,03a05508f9a2899dcd3e689fb47ed900,0.8289,0.8298,0.7459,0.9930,[]
1637,1da1ba1081749ce54622f0590bfde704,0.8288,0.8295,0.7461,0.9930,[]
...,...,...,...,...,...,...
2476,65a7098260880ec30c1da27059f8f95f,0.3092,0.7712,0.0000,0.0037,[]
2483,c0daf1c78478267109aaf5422608db05,0.3092,0.7712,0.0000,0.0037,[]
2477,d488b1da51fcff2bf3eb3b5077804cb6,0.3092,0.7712,0.0000,0.0037,[]
1808,6151c496a834491ebf776043463768e0,0.2594,0.6449,0.0000,0.0072,[]


In [10]:
metronome_bots.describe()

,bot_suspicion_score,centrality_rank,clique_score,volume_score
count,2485.000000,2485.000000,2485.000000,2485.00000
mean,0.729997,0.771160,0.743526,0.62061
std,0.064036,0.033537,0.055701,0.26652
min,0.176500,0.434800,0.000000,0.00360
25%,0.697800,0.757100,0.727100,0.44270
50%,0.736900,0.770700,0.747400,0.66580
75%,0.780100,0.789300,0.764100,0.85880
max,0.830800,1.000000,1.000000,1.00000


In [11]:
filtered_metronome_bots = metronome_bots[
    metronome_bots['bot_suspicion_score'] >
    metronome_bots['bot_suspicion_score'].quantile(0.8)
]
filtered_metronome_bots

,user_id,bot_suspicion_score,centrality_rank,clique_score,volume_score,labels
38,96dd9aa3ec0b34b3960f2e1e939a8d64,0.8308,0.8351,0.7418,1.0000,[]
1620,07916b5f9d395b4ba287939d7662b94d,0.8289,0.8296,0.7461,0.9930,[]
137,6106f6a23e4332847cbafa9f5bcccdbd,0.8289,0.8297,0.7459,0.9929,[]
1593,03a05508f9a2899dcd3e689fb47ed900,0.8289,0.8298,0.7459,0.9930,[]
1637,1da1ba1081749ce54622f0590bfde704,0.8288,0.8295,0.7461,0.9930,[]
...,...,...,...,...,...,...
520,43be71b62e7cb82a91cb31fd6cf728fc,0.7852,0.7613,0.7559,0.8918,[]
417,5266ce2f84b7f8969776d4f8439ffa99,0.7851,0.7589,0.7597,0.8883,[]
1143,0f85e0d82a954246f01535a362bedf72,0.7851,0.7659,0.7883,0.8173,[]
993,a4de662ef0a8c1693e323d3c217a6fbd,0.7851,0.7631,0.7891,0.8209,[]


## Avaliando Semelhança entre Técnicas de Detecção

In [12]:
metronome_susp = set(filtered_metronome_bots['user_id'])
hourly_susp = set(filtered_hourly_bots['user_id'])
rapid_susp = set(filtered_rapid_bots['user_id'])

In [13]:
overlap_mh = metronome_susp & hourly_susp
overlap_mr = metronome_susp & rapid_susp
overlap_hr = hourly_susp & rapid_susp

len(overlap_mh), len(overlap_mr), len(overlap_hr)

(86, 10, 89)

In [14]:
def jaccard(a, b):
    return len(a & b) / len(a | b) if (a | b) else 0

j12 = jaccard(metronome_susp, hourly_susp)
j13 = jaccard(metronome_susp, rapid_susp)
j23 = jaccard(hourly_susp, rapid_susp)

j12, j13, j23

(0.09662921348314607, 0.01718213058419244, 0.18350515463917524)

In [15]:
def overlap_ratio(a, b):
    return len(a & b) / min(len(a), len(b)) if min(len(a), len(b)) else 0

o12 = overlap_ratio(metronome_susp, hourly_susp)
o13 = overlap_ratio(metronome_susp, rapid_susp)
o23 = overlap_ratio(rapid_susp, hourly_susp)

o12, o13, o23

(0.17954070981210857, 0.10526315789473684, 0.9368421052631579)

In [16]:
highest_confident = metronome_susp & hourly_susp & rapid_susp
len(highest_confident)

8

In [17]:
concatenated = metronome_susp | hourly_susp | rapid_susp
len(concatenated)

894

In [18]:
import pandas as pd

pd.DataFrame({'user_id': list(highest_confident)}).to_csv(
    '../data/highest_confident_bots.csv', index=False
)

pd.DataFrame({'user_id': list(concatenated)}).to_csv(
    '../data/all_suspected_bots.csv', index=False
)